In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train = '/content/drive/MyDrive/Colab Notebooks/output/train'
test = '/content/drive/MyDrive/Colab Notebooks/output/test'
val = '/content/drive/MyDrive/Colab Notebooks/output/val'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_shape = (224,224,3)
image_gen = ImageDataGenerator(rotation_range=20,
                               rescale=1/255,
                               horizontal_flip=True,
                               fill_mode='nearest'
                              )


In [ ]:
image_gen.flow_from_directory(train)
image_gen.flow_from_directory(test)
image_gen.flow_from_directory(val)

In [ ]:
train_image_gen = image_gen.flow_from_directory(train,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=32,
                                               class_mode='categorical')

test_image_gen = image_gen.flow_from_directory(test,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=32,
                                               class_mode='categorical',shuffle=True)

val_image_gen = image_gen.flow_from_directory(val,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=32,
                                               class_mode='categorical',shuffle=False)

In [ ]:
train_image_gen.class_indices

## INCEPTIONV3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

In [ ]:
# Load the pre-trained InceptionV3 model without the top layer
base_modelv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))


In [ ]:
x = base_modelv3.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions1 = Dense(8, activation='softmax')(x)


In [ ]:
modelv3 = Model(inputs=base_modelv3.input, outputs=predictions1)

In [ ]:
for layer in base_modelv3.layers[:1]:
    layer.trainable = False


In [ ]:
modelv3.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
historyv3 = modelv3.fit(x=train_image_gen,
                      epochs=30,
                      validation_data=val_image_gen)


In [ ]:
test_historyv3 = modelv3.evaluate(x=test_image_gen)

In [ ]:
print(historyv3.history.keys())
#  "Accuracy"
plt.plot(historyv3.history['accuracy'])
plt.plot(historyv3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(historyv3.history['loss'])
plt.plot(historyv3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
modelv3.save('/content/drive/MyDrive/Colab Notebooks/Endo-inception97.h5')

## DENSENET121


In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121


In [ ]:
# Define the number of classes
num_classes = len(train_image_gen.class_indices)

# Load the pre-trained model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))



In [ ]:
# Add a new classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)



In [ ]:
# Create the final model
dense_model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
dense_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
historyd = dense_model.fit(train_image_gen, epochs=30, validation_data=val_image_gen)



In [ ]:
test_historyd = dense_model.evaluate(x=test_image_gen)

In [ ]:
print(historyd.history.keys())
#  "Accuracy"
plt.plot(historyd.history['accuracy'])
plt.plot(historyd.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(historyd.history['loss'])
plt.plot(historyd.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
dense_model.save('/content/drive/MyDrive/Colab Notebooks/Endo-DenseNet97.h5')

## VGG-16

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
basic_vgg = VGG16(input_shape=(224,224,3),
                 include_top=False,
                 weights='imagenet')



In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout

In [ ]:
x=Flatten()(basic_vgg.output)
x=Dense(512,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(8,activation='softmax')(x)

In [ ]:
modelvgg =tf.keras.Model(basic_vgg.input,x)
modelvgg.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
historyvgg = modelvgg.fit(x=train_image_gen,
                      epochs=25,
                      validation_data=val_image_gen)


In [ ]:
test_metrics = modelvgg.evaluate(x=test_image_gen)

In [ ]:
# Print test set loss and accuracy
print("Test Loss:", test_metrics[0])
print("Test Accuracy:", test_metrics[1])


In [ ]:
# Plot "Accuracy"
plt.plot(history_vgg.history['accuracy'])
plt.plot(history_vgg.history['val_accuracy'])
plt.title('Model Accuracy Over Epochs (VGG16)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot "Loss"
plt.plot(history_vgg.history['loss'])
plt.plot(history_vgg.history['val_loss'])
plt.title('Model Loss Over Epochs (VGG16)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()